In [39]:
import cv2
import numpy as np
import time
import math
import sys
import torch
from time import time

video_path = './video/test2.mp4'
srt_path = './video/test2.srt'

# pretrained (사람만 검출 - 0번째 class 번호)
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model.classes = [0]
# custom 학습
# model = torch.hub.load('.', 'custom', path='./runs/train/exp2/weights/best.pt', source='local')

def get_hmsms(dt):
    ms = int((dt - int(dt))*1000)
    dt = int(dt)
    hh, dt = divmod(dt, 3600)
    mm, ss = divmod(dt, 60)
    return (hh, mm, ss, ms)

# Video input
cap = cv2.VideoCapture(video_path)

if not cap.isOpened:
    print('--(!)Error opening video capture')
    sys.exit(1)

Ghh = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
Gww = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
FPS = cap.get(cv2.CAP_PROP_FPS)
NUM = cap.get(cv2.CAP_PROP_FRAME_COUNT)

Ghh, Gww = int(Ghh), int(Gww)

print(f"video_path = {video_path}")
print(f"\nvideo resolution: {[Gww, Ghh]}, fps:{FPS}")

rr = input("start?(y/n):")
if(rr.upper() == 'N'):
    sys.exit(1)

# x, y, w, h = rect
x = int(Gww*0.1)
y = int(Ghh*0.1)
w = int(Gww*0.8)
h = int(Ghh*0.8)    

# 필요 변수 선언
i = 0 # frame count & 저장 리스트 비교용
tmp = 0 # 첫번째 detection 구분
notcnt = 0 # 아무것도 검출이 안되는 상황 대비, 일정 카운트 이상일 시 원본 frame 크기로 변경
case_not = 0 # 검출이 안되다가 갑자기 검출될 시 발생하는 예외 제거
event = 0 # 이벤트 상황 구분
j = 1 # 프레임 별 시간 계산용
check = 1 # 자막 append 간격 지정용
time_tmp = 0 # 시간 cnt 시작
event_cnt = 0 # 이벤트 초기화

#FPS = 15
GDt = 1.0/FPS
t1 = 0.0

# 프레임 간 좌표 계산을 위한 저장용 list 생성, 초기값 append
PX = []
PY = []
X_min = []
X_max = []
Y_min = []
Y_max = []
X_min.append(x)
X_max.append(x+w)
Y_min.append(y)
Y_max.append(y+h)

PX.append((X_max[i] + X_min[i])/2)
PY.append((Y_max[i] + Y_min[i])/2)

file_srt = open(srt_path, 'w')

tt_i = time()

while cap.isOpened():
    tt0 = time()
    success, img = cap.read()
    if not success:
        print("Ignoring empty camera frame.")
        break
    
    # 첫번째 detection
    if tmp == 0:
        results = model(img)
        cv2.imshow('Falling Detection', img)
        
        # 사람이 검출될 시
        if len(results.xyxy[0]) > 0:
            x = int(results.pandas().xyxy[0].xmin[0])
            y = int(results.pandas().xyxy[0].ymin[0])
            w = int(results.pandas().xyxy[0].xmax[0]) - x
            h = int(results.pandas().xyxy[0].ymax[0]) - y
            notcnt = 0
            if case_not > 0:
                case_not-=1

            # 임의로 박스 주변 crop 범위 지정
            x_ = int((2*x-w) / 2)
            y_ = int((2*y-h) / 2)
            w_ = int(2*w)
            h_ = int(2*h)
            
            if x_ <= 0: x_ = 0
            if y_ <= 0: y_ = 0
            
            xw = x_ + w_
            yh = y_ + h_

            if xw >= Gww: 
                xw = Gww
            if yh >= Ghh: 
                yh = Ghh

            ##### 낙상 판단 방향
            X_min.append(x)
            X_max.append(x+w)
            Y_min.append(y)
            Y_max.append(y+h)
        
            tmp += 1
        
        if time_tmp == 0:
            hh1 = 0
            mm1 = 0
            ss1 = 0
            ms1 = 0
            hh2, mm2, ss2, ms2 = get_hmsms(GDt);
            time_tmp+=1
            
        qq = '정상'
        t1 = t1 + GDt
        hh1, mm1, ss1, ms1 = get_hmsms(t1)
        hh2, mm2, ss2, ms2 = get_hmsms(t1 + GDt);
        
        print("%d\n%02d:%02d:%02d,%03d --> %02d:%02d:%02d,%03d\n%s\n" % (j, hh1, mm1, ss1, ms1, hh2, mm2, ss2, ms2, qq), file=file_srt)
        j+=1
        
        key_pressed = cv2.waitKey(1) & 0xFF
        if key_pressed == ord('q'):
            break
    
    # crop된 frame 안에서 detection 진행
    if tmp > 0:
        image = img[y_:yh, x_:xw]
        # image = img
        results = model(image)
        
        t1 = t1 + GDt
        hh1, mm1, ss1, ms1 = get_hmsms(t1)
        hh2, mm2, ss2, ms2 = get_hmsms(t1 + GDt);
        
        # 축소된 프레임 안에서의 좌표 설정, 조정된 값 계산
        if len(results.xyxy[0]) > 0:
            x = int(results.pandas().xyxy[0].xmin[0]) + x_
            y = int(results.pandas().xyxy[0].ymin[0]) + y_
            w = int(results.pandas().xyxy[0].xmax[0]) + x_ - x
            h = int(results.pandas().xyxy[0].ymax[0]) + y_ - y
            notcnt = 0
            if case_not > 0:
                case_not-=1
        elif len(results.xyxy[0]) == 0 and i % 10 == 0: # 검출 안될 시 image 크기 조금씩 증가, 오래 검출 안될 시 원본 프레임 크기 복구
            x-=1
            y-=1
            w+=2
            h+=2
            notcnt+=1
            if notcnt % 7 == 0:
                x = int(Gww*0.1)
                y = int(Ghh*0.1)
                w = int(Gww*0.8)
                h = int(Ghh*0.8)
                case_not = 5
        
        # 임의로 박스 주변 crop 범위 지정
        x_ = int((2*x-w) / 2)
        y_ = int((2*y-h) / 2)
        w_ = int(2*w)
        h_ = int(2*h)
        xw = x_+w_
        yh = y_+h_

        if x_ <= 0: x_ = 0
        if y_ <= 0: y_ = 0
        if xw >= Gww: xw = Gww
        if yh >= Ghh: yh = Ghh
        
        print("box info :", x,y,x+w,y+h)
        
        # 바운딩 박스 좌표 값 저장
        X_min.append(x)
        X_max.append(x+w)
        Y_min.append(y)
        Y_max.append(y+h)
        
        ### 낙상 판단 방향
        case_d = 0
        if ((X_max[i] + Y_min[i])/2 > (X_min[i] + Y_max[i])/2) and i>0:
            #좌측낙사 case = 3
            if X_min[i] < X_min[i-1] and X_max[i] < X_max[i-1] and Y_min[i] > Y_min[i-1] and Y_max[i] > Y_max[i-1]:
                case_d = 3

            #우측낙사 case = 3
            elif X_min[i] > X_min[i-1] and X_max[i] > X_max[i-1] and Y_min[i] > Y_min[i-1] and Y_max[i] > Y_max[i-1]:
                case_d = 3         

            #전방낙사 case = 1
            elif Y_min[i] > Y_min[i-1] and Y_max[i] > Y_max[i-1]:
                case_d = 1

            #후방낙사 case = 2
            elif Y_min[i] > Y_min[i-1] :
                case_d = 2
        
        ### 속력 낙상 판단
        case_v = 0
        PX.append((X_max[i] + X_min[i])/2)
        PY.append((Y_max[i] + Y_min[i])/2)
        Pixel2cm = (Y_max[i-1]+Y_min[i-1])/170

        a = (X_max[i]+X_min[i])/2 - (X_max[i-1]+X_min[i-1])/2
        b = (Y_max[i]+Y_min[i])/2 - (Y_max[i-1]+Y_min[i-1])/2

        #속력 구하기
        V = (math.sqrt(a**2 + b**2) * Pixel2cm)/(1/2)

        if V >= 350 and V < 3000:
            if X_min[i-2] != int(Gww*0.1) and Y_min[i-2] != int(Ghh*0.1) and X_max[i-2] != int(Gww*0.9) and Y_max[i-2] != int(Ghh*0.9):
                case_v = 1
        elif V >= 300 and V < 350:
            case_v = 2
        else:
            case_v = 0
        i+=1
        
        # 정상일 때 자막 저장 & 이상 상황 발생 시 event 값 부여
        if event == 0:
            if case_not == 0 and case_d == 1 and case_v == 1:
                event = 1
            elif case_not == 0 and case_d == 2 and case_v == 1:
                event = 2
            elif case_not == 0 and case_d == 3 and case_v == 1:
                event = 3
            elif case_not == 0 and case_d > 0 and case_v == 2:
                event = 4
            else:
                print("<<<정상>>>")
                print("속도 :", V)
                qq = '정상'
        
        # 전방 낙상 경우
        if event == 1:
            print("<<<앞으로 넘어짐!>>>")
            print("속도 :", V)
            qq = '앞으로 넘어짐!'
            event_cnt+=1

        # 후방 낙상 경우
        if event == 2:
            print("<<<뒤로 넘어짐!>>>")
            print("속도 :", V)
            qq = '뒤로 넘어짐!'
            event_cnt+=1
                
        # 측면 낙상 경우
        if event == 3:
            print("<<<옆으로 넘어짐!>>>")
            print("속도 :", V)
            qq = '옆으로 넘어짐!'
            event_cnt+=1

        # 낙상이 의심되는 상황
        if event == 4:
            print("<<<낙상 의심>>>")
            print("속도 :", V)
            qq = '낙상 의심'
            event = 0
        
        # 10초 후 상태 초기화
        if event_cnt == 150:
            event = 0
            event_cnt = 0

        j+=1
        check+=1
        
        print("%d\n%02d:%02d:%02d,%03d --> %02d:%02d:%02d,%03d\n%s\n" % (j, hh1, mm1, ss1, ms1, hh2, mm2, ss2, ms2, qq), file=file_srt)
        
        for k in range(len(results.xyxy[0])):
            conf = results.pandas().xyxy[0].confidence[0]*100
            label = "person: {:.2f}".format(conf)
            color_g = (0, 255, 0)
            cv2.rectangle(image, (int(results.pandas().xyxy[0].xmin[0]), int(results.pandas().xyxy[0].ymin[0])), (int(results.pandas().xyxy[0].xmax[0]), int(results.pandas().xyxy[0].ymax[0])), color_g, 2)
            cv2.putText(image, label, (int(results.pandas().xyxy[0].xmin[0]), int(results.pandas().xyxy[0].ymin[0] - 5)), cv2.FONT_HERSHEY_DUPLEX, 1, color_g, 1)
        
        #결과 show
        cv2.imshow('Falling Detection', image)

        key_pressed = cv2.waitKey(1) & 0xFF
        if key_pressed == ord('q') or key_pressed == 27:
            break
        
        print(f"frame time: {(time() - tt0):.4f}, elapsed: {(time() - tt_i):.4f}----------")
                
file_srt.close()
        
cap.release()
cv2.destroyAllWindows()

Using cache found in C:\Users\SDH-LAB/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-7-20 Python-3.8.13 torch-1.12.0 CUDA:0 (GeForce GTX 1060 3GB, 3072MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


video_path = ./video/test2.mp4

video resolution: [1920, 1080], fps:30.0117491479011
start?(y/n):y
box info : 600 707 967 1009
<<<정상>>>
속도 : 7342.421210649936
frame time: 0.1551, elapsed: 0.1572----------
box info : 606 708 966 994
<<<정상>>>
속도 : 4435.758920878393
frame time: 0.0509, elapsed: 0.2080----------
box info : 605 708 968 1000
<<<낙상 의심>>>
속도 : 326.84996558053973
frame time: 0.0582, elapsed: 0.2662----------
box info : 598 709 966 999
<<<정상>>>
속도 : 150.05984688469738
frame time: 0.0470, elapsed: 0.3132----------
box info : 611 707 967 1007
<<<정상>>>
속도 : 60.899187215103346
frame time: 0.0480, elapsed: 0.3612----------
box info : 608 709 968 989
<<<정상>>>
속도 : 90.4235294117647
frame time: 0.0479, elapsed: 0.4092----------
box info : 606 708 968 1001
<<<정상>>>
속도 : 153.03224076253596
frame time: 0.0459, elapsed: 0.4551----------
box info : 610 708 967 1002
<<<정상>>>
속도 : 162.57305624216133
frame time: 0.0493, elapsed: 0.5043----------
box info : 605 707 968 989
<<<정상>>>
속도 : 111.6718

frame time: 0.0636, elapsed: 4.8311----------
box info : 571 699 858 998
<<<정상>>>
속도 : 271.84469831137045
frame time: 0.0635, elapsed: 4.8946----------
box info : 601 700 859 990
<<<정상>>>
속도 : 278.5983647800746
frame time: 0.0638, elapsed: 4.9585----------
box info : 581 700 856 992
<<<정상>>>
속도 : 382.6709935092499
frame time: 0.0630, elapsed: 5.0215----------
box info : 603 700 861 998
<<<정상>>>
속도 : 317.24413933851747
frame time: 0.0625, elapsed: 5.0840----------
box info : 614 700 859 1003
<<<정상>>>
속도 : 229.50988097929036
frame time: 0.0630, elapsed: 5.1471----------
box info : 564 702 860 1003
<<<정상>>>
속도 : 275.2847509718747
frame time: 0.0639, elapsed: 5.2110----------
box info : 610 699 859 985
<<<정상>>>
속도 : 102.83517634938781
frame time: 0.0639, elapsed: 5.2749----------
box info : 587 701 858 1004
<<<정상>>>
속도 : 491.27341928029193
frame time: 0.0638, elapsed: 5.3387----------
box info : 605 698 856 1002
<<<정상>>>
속도 : 498.04891471951544
frame time: 0.0625, elapsed: 5.4012----------

box info : 582 701 858 986
<<<정상>>>
속도 : 109.80588235294118
frame time: 0.0634, elapsed: 9.7946----------
box info : 587 703 848 987
<<<정상>>>
속도 : 60.72028243480046
frame time: 0.0632, elapsed: 9.8578----------
box info : 573 701 848 998
<<<정상>>>
속도 : 107.91202876212223
frame time: 0.0799, elapsed: 9.9377----------
box info : 584 702 853 990
<<<정상>>>
속도 : 57.863622627082485
frame time: 0.0619, elapsed: 9.9996----------
box info : 589 699 854 999
<<<정상>>>
속도 : 165.45415112522102
frame time: 0.0632, elapsed: 10.0627----------
box info : 579 701 852 990
<<<정상>>>
속도 : 174.5397610881029
frame time: 0.0630, elapsed: 10.1258----------
box info : 585 703 853 996
<<<정상>>>
속도 : 84.453506383598
frame time: 0.0633, elapsed: 10.1891----------
box info : 571 698 855 997
<<<정상>>>
속도 : 138.7609993769751
frame time: 0.0634, elapsed: 10.2524----------
box info : 581 696 860 988
<<<정상>>>
속도 : 105.73868570196643
frame time: 0.0629, elapsed: 10.3153----------
box info : 586 700 845 986
<<<정상>>>
속도 : 126.41

box info : 616 668 867 975
<<<정상>>>
속도 : 0.0
frame time: 0.0319, elapsed: 14.4412----------
box info : 616 668 867 975
<<<정상>>>
속도 : 0.0
frame time: 0.0308, elapsed: 14.4721----------
box info : 615 667 868 976
<<<정상>>>
속도 : 0.0
frame time: 0.0470, elapsed: 14.5191----------
box info : 615 667 868 976
<<<정상>>>
속도 : 0.0
frame time: 0.0314, elapsed: 14.5505----------
box info : 615 667 868 976
<<<정상>>>
속도 : 0.0
frame time: 0.0322, elapsed: 14.5827----------
box info : 615 667 868 976
<<<정상>>>
속도 : 0.0
frame time: 0.0310, elapsed: 14.6137----------
box info : 615 667 868 976
<<<정상>>>
속도 : 0.0
frame time: 0.0320, elapsed: 14.6457----------
box info : 631 667 881 977
<<<정상>>>
속도 : 0.0
frame time: 0.0470, elapsed: 14.6927----------
box info : 632 666 882 979
<<<정상>>>
속도 : 0.0
frame time: 0.0632, elapsed: 14.7559----------
box info : 633 666 884 979
<<<정상>>>
속도 : 280.4430539435596
frame time: 0.0626, elapsed: 14.8185----------
box info : 633 666 884 979
<<<정상>>>
속도 : 21.62409267652738
frame t

frame time: 0.0319, elapsed: 18.4124----------
box info : 667 639 948 940
<<<정상>>>
속도 : 0.0
frame time: 0.0488, elapsed: 18.4613----------
box info : 667 639 948 940
<<<정상>>>
속도 : 0.0
frame time: 0.0296, elapsed: 18.4918----------
box info : 667 639 948 940
<<<정상>>>
속도 : 195.6873614355448
frame time: 0.0317, elapsed: 18.5235----------
box info : 667 639 948 940
<<<정상>>>
속도 : 0.0
frame time: 0.0469, elapsed: 18.5704----------
box info : 667 639 948 940
<<<정상>>>
속도 : 0.0
frame time: 0.0313, elapsed: 18.6017----------
box info : 672 645 963 941
<<<정상>>>
속도 : 0.0
frame time: 0.0641, elapsed: 18.6658----------
box info : 672 645 963 941
<<<정상>>>
속도 : 0.0
frame time: 0.0320, elapsed: 18.6977----------
box info : 672 645 963 941
<<<정상>>>
속도 : 196.81417728563875
frame time: 0.0320, elapsed: 18.7297----------
box info : 672 645 963 941
<<<정상>>>
속도 : 0.0
frame time: 0.0316, elapsed: 18.7613----------
box info : 671 644 964 942
<<<정상>>>
속도 : 0.0
frame time: 0.0318, elapsed: 18.7931----------
box 

box info : 715 649 1050 958
<<<옆으로 넘어짐!>>>
속도 : 57.46421543328788
frame time: 0.0789, elapsed: 22.1643----------
box info : 713 649 1046 957
<<<옆으로 넘어짐!>>>
속도 : 55.29114384994485
frame time: 0.0631, elapsed: 22.2274----------
box info : 716 644 1043 956
<<<옆으로 넘어짐!>>>
속도 : 0.0
frame time: 0.0629, elapsed: 22.2903----------
box info : 713 652 1045 955
<<<옆으로 넘어짐!>>>
속도 : 57.49999638934846
frame time: 0.0628, elapsed: 22.3531----------
box info : 714 648 1034 950
<<<옆으로 넘어짐!>>>
속도 : 56.682352941176475
frame time: 0.0629, elapsed: 22.4160----------
box info : 714 648 1034 950
<<<옆으로 넘어짐!>>>
속도 : 66.55122646461625
frame time: 0.0320, elapsed: 22.4481----------
box info : 713 641 1041 961
<<<옆으로 넘어짐!>>>
속도 : 127.17631672733795
frame time: 0.0470, elapsed: 22.4951----------
box info : 719 655 1030 950
<<<옆으로 넘어짐!>>>
속도 : 0.0
frame time: 0.0634, elapsed: 22.5585----------
box info : 708 648 1039 961
<<<옆으로 넘어짐!>>>
속도 : 67.784363978723
frame time: 0.0636, elapsed: 22.6221----------
box info : 